<a href="https://colab.research.google.com/github/BitByBitAIGroup/Lab-Problem-1/blob/main/problem1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np

class QuizMDP:
    def __init__(self, rewards, probabilities):
        # Rewards for each question
        self.q_rewards = rewards
        # Probabilities of answering a question correctly
        self.q_probs = probabilities
        # Total number of questions
        self.num_questions = len(rewards)
        # Total number of states (questions + end state)
        self.num_states = self.num_questions + 1
        # Starting state
        self.start_state = 0
        # End state
        self.end_state = self.num_states - 1

    # Get the probabilities of transitioning to the next state
    def get_transition_probs(self, state, action):
        if state == self.end_state:
            # If already in the end state, stay there with probability 1
            return [(self.end_state, 1.0)]
        elif action == 1:
            # If answering a question, transition to the next state with probability q_probs[state]
            # or go to the end state with probability 1 - q_probs[state]
            return [(state + 1, self.q_probs[state]), (self.end_state, 1 - self.q_probs[state])]
        else:
            # If passing, go directly to the end state with probability 1
            return [(self.end_state, 1.0)]

    # Get the rewards for transitioning to the next state
    def get_rewards(self, state, action, next_state):
        if next_state == self.end_state:
            # If already in the end state, get no reward
            return 0
        elif action == 1:
            # If answering a question, get the reward for the current state
            return self.q_rewards[state]
        else:
            # If passing, get no reward
            return 0

# Use value iteration to find the optimal policy
def find_optimal_policy(mdp, gamma=1.0, theta=1e-6):
    # Initialize the value function to zero
    value_function = np.zeros(mdp.num_states)

    while True:
        # Initialize the change in V to zero
        max_change_in_value_function = 0

        # Update the value function for each state
        for state in range(mdp.num_states):
            old_value_function = value_function[state]

            # Get the maximum expected value for each action
            expected_values_for_actions = []
            for action in range(2):
                expected_value_for_action = 0
                for next_state, transition_probability in mdp.get_transition_probs(state, action):
                    reward = mdp.get_rewards(state, action, next_state)
                    expected_value_for_action += transition_probability * (reward + gamma * value_function[next_state])
                expected_values_for_actions.append(expected_value_for_action)

            value_function[state] = max(expected_values_for_actions)

            # Update delta to keep track of the largest change in V
            max_change_in_value_function = max(max_change_in_value_function, abs(old_value_function - value_function[state]))

        # Check for convergence
        if max_change_in_value_function < theta:
            break

    return value_function
